# HOMEWORK 3
LEYLA ALAS 090190739

With this homework, we are going to learn how to analyse a text from html format of news. The needed libraries and the functions are in the cell below. *requests* is for reading html url's. *nltk* is for analysing the text word and sentence level and it is used to evaluate polarity of each sentences from a text. *regex* was used for finding where sentences are ended. This homework we are using *spacy*'s NER. and lastly *en_core_web_sm* is for English vocabulary.

In [98]:
import requests
import nltk
import regex as re
import numpy as np
import spacy
import pandas as pd
import en_core_web_sm

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from bs4 import BeautifulSoup
from xmltodict import parse

+ The cell below contain the functions that I used the most during this homework.
+ **getSubjectGuardian** take the subject and return it as dictionary format and we use **getText** with the returned dictionary's link element.
+ I used the **unique** function to make the list elements unique.

In [111]:
def getSubjectGuardian(subject):
    with requests.get(f'https://www.theguardian.com/{subject}/rss') as link:
        raw = parse(link.text)
    return raw['rss']['channel']['item']

fashion = getSubjectGuardian('fashion')

def getText(url):
    with requests.get(url) as link:
        raw = BeautifulSoup(link.content,'html.parser')
    return ' '.join([x.text for x in raw.find_all('p')])

def unique(lisst):
    unique_list = []
    for x in lisst:
        if x not in unique_list:
            unique_list.append(x)
    return unique_list

## Q1

I used FOX's RSS feed for this question. Here is example for the links;
+ National: https://moxie.foxnews.com/feedburner/national.xml
+ World: https://moxie.foxnews.com/feedburner/world.xml
+ Sports: https://moxie.foxnews.com/feedburner/sports.xml

Then, I turned html url's to text.

In [100]:
def getSubjectFOX(subject):
    with requests.get(f'https://moxie.foxnews.com/feedburner/{subject}.xml') as link:
        raw = parse(link.text)
    return raw['rss']['channel']['item']

def getText1(url):
    with requests.get(url) as link:
        raw = BeautifulSoup(link.content,'html.parser')
    return ' '.join([x.text for x in raw.find_all('p')])

latest = getSubjectFOX('latest')
textfox = getText(latest[3]['link'])
textguardian = getText(fashion[2]['link'])

Here is the functions for question 1. 

In [101]:
def getSummary(text,k):
    sentences = processText(text)
    matrix = getMatrix(sentences)
    projection = PCA(n_components=1)
    weights = projection.fit_transform(matrix.toarray())
    res = list(zip(weights.transpose()[0],range(112),sentences))
    tmp = sorted(res,key=lambda x: x[0],reverse=True)[:k]
    return sorted(tmp, key=lambda x: x[1])

def processText(text):
    sentences = re.split(r'[\.\?\!]',text)
    return [re.sub(r'[^\w\s]','',x.lower()) for x in sentences]

def getMatrix(sentences):
    vectorizer = CountVectorizer()
    return vectorizer.fit_transform(sentences)

def getKeywords(text,sw,k):
    sentences = processText(text)
    
    vectorizer = CountVectorizer(stop_words=sw)
    matrix = vectorizer.fit_transform(sentences)
    words = vectorizer.get_feature_names()
    
    projection = PCA(n_components=1)
    tmp = projection.fit_transform(matrix.transpose().toarray())
    weights = tmp.transpose()[0]
    
    return sorted(zip(weights,words),key=lambda x: x[0], reverse=True)[:k]

I introduced the stopwords in English to the system, and I removed the k-highest from the text and printed them on the screen.
+ The first one is from Guardian and the second one is from FOX.
+ I used fashion's the second news for Guardian and latest's the third new for FOX.

In [105]:
swEN = stopwords.words('english')
getKeywords(textguardian,swEN,15)

[(3.2519912366433217, 'bag'),
 (1.4258822827493554, 'baguette'),
 (1.0338630724556481, 'designed'),
 (0.9446542513943269, 'fendi'),
 (0.7092838113582342, 'name'),
 (0.7032832122194975, 'arm'),
 (0.7032832122194973, 'borrellipersson'),
 (0.7032832122194973, 'borrows'),
 (0.7032832122194973, 'crusty'),
 (0.7032832122194973, 'easily'),
 (0.7032832122194973, 'french'),
 (0.7032832122194973, 'laird'),
 (0.7032832122194973, 'loaf'),
 (0.7032832122194973, 'long'),
 (0.7032832122194973, 'nestles')]

In [107]:
text_tokens = word_tokenize(textguardian)
tokens_without_sw = [word for word in text_tokens if not word in swEN]
print(tokens_without_sw)

['Wrapuette', ',', 'inspired', 'Baguette', 'bag', '2000s', ',', 'taps', 'logo', 'mania', '‘', 'irony', '’', '-loving', 'fashion', 'crowd', 'The', 'American', 'fast', 'food', 'chain', 'KFC', 'launched', '£198', 'insulated', 'leather', 'handbag', 'British', 'market', ',', 'designed', 'able', 'hold', 'one', 'chicken', 'chain', '’', 'Twister', 'Wraps', '.', 'The', '“', 'Wrapuette', '”', ',', 'inspired', 'Baguette', 'bag', '2000s', 'popularised', 'fashion', 'brands', 'Fendi', ',', 'Dior', 'Gucci', ',', 'sold', 'limited', 'edition', 'via', 'KFC', '’', 'online', 'shop', '.', 'According', 'website', ',', 'proceeds', 'go', 'KFC', 'Foundation', ',', 'says', ',', '“', 'supports', 'grassroots', 'non-profit', 'organisations', 'empower', 'young', 'people', 'across', 'UK', ',', 'helping', 'fulfil', 'potential', 'build', 'positive', 'future', '”', '.', 'The', 'handbag', 'comes', 'pebbled', 'textured', 'leather', 'KFC', '’', 'signature', 'tomato', 'red', ',', 'features', 'monochrome', 'image', 'Colonel

In [103]:
getKeywords(textfox,swEN,15)

[(2.6646603555354593, 'true'),
 (1.9578256243517402, 'use'),
 (1.724337222384731, 'call'),
 (1.724337222384731, 'false'),
 (1.2811617852066457, 'disinformation'),
 (1.0253070870116425, 'concept'),
 (0.9388946356145317, 'jankowicz'),
 (0.9388946356145317, 'nina'),
 (0.801960820387931, 'reject'),
 (0.7840140892340034, 'abused'),
 (0.7840140892340027, 'figure'),
 (0.7840140892340027, 'hunters'),
 (0.7840140892340027, 'inquiry'),
 (0.7840140892340027, 'know'),
 (0.7840140892340027, 'laptop')]

In [104]:
text_tokens = word_tokenize(textfox)
tokens_without_sw = [word for word in text_tokens if not word in swEN]
print(tokens_without_sw)

['This', 'material', 'may', 'published', ',', 'broadcast', ',', 'rewritten', ',', 'redistributed', '.', '©2022', 'FOX', 'News', 'Network', ',', 'LLC', '.', 'All', 'rights', 'reserved', '.', 'Quotes', 'displayed', 'real-time', 'delayed', 'least', '15', 'minutes', '.', 'Market', 'data', 'provided', 'Factset', '.', 'Powered', 'implemented', 'FactSet', 'Digital', 'Solutions', '.', 'Legal', 'Statement', '.', 'Mutual', 'Fund', 'ETF', 'data', 'provided', 'Refinitiv', 'Lipper', '.', 'Compact', 'Magazine', 'founder', 'Sohrab', 'Ahmari', 'unpacks', 'Biden', 'administration', "'s", 'Disinformation', 'Governance', 'Board', 'consequences', 'nation', '.', 'Compact', 'Magazine', 'founder', 'Sohrad', 'Ahmari', 'said', '``', 'Fox', '&', 'Friends', "''", 'Friday', 'Biden', 'administration', 'putting', 'forth', '``', 'politicized', "''", '``', 'dangerous', "''", 'policy', 'toward', '``', 'disinformation', '.', "''", 'Ahmari', 'said', 'DHS', "'", 'Disinformation', 'Governance', 'Board', 'new', 'director',

## Q2

This function takes the url of the text as input. Then it loads the English vocabulary and makes the words in the text unique. Then, I group these words inside the function.

In [116]:
def q2func(url):
    text = getText(url)
        
    NER = spacy.load("en_core_web_sm")
    ress = NER(text)
    lisst = [(word.text, word.label_) for word in ress.ents]
    b = unique(lisst)

    proper = []
    country = []
    corporation = []
    
    for x in range(len(b)):

        if b[x][1] == 'PERSON' or b[x][1] == 'LOC' or b[x][1] == 'WORK_OF_ART':
            proper.append(b[x][0])
        elif b[x][1] == 'GPE':
            country.append(b[x][0])
        elif b[x][1] == 'ORG' or b[x][1] == 'PRODUCT':
            corporation.append(b[x][0])
    new = {'Proper Names' : proper, 'Country Names' : country, 'Corporation Names' : corporation}
    return new

In [119]:
q2func(fashion[2]['link'])

{'Proper Names': ['Baguette',
  'Wrapuette',
  'Gucci',
  'Savile Row',
  'Natalie Hartley',
  'Silvia Venturini Fendi',
  'Laird Borrelli-Persson',
  'Chillie London',
  'Wrapuette',
  'Gen',
  'Jane Collins'],
 'Country Names': ['UK', 'Hartley', 'Baguette'],
 'Corporation Names': ['Wrapuette',
  'Fendi',
  'the KFC Foundation',
  'Vogue',
  'Hartley',
  'eBay']}

## Q3

In [203]:
text = getText(fashion[0]['link'])

With q3func I first found the most negative and most positive scores in the text. Then, I found which sentences are equal to these points in the list format, where we translated the text, with the search function. Finally, sentences return with their scores from q3func.

In [162]:
def search(values, polarity, tip):
    for i in range(len(values)):
        if values[i][1][tip] == polarity:
            return i
    return i

def q3func(text):
    analyzer = SentimentIntensityAnalyzer()
    sentences = sent_tokenize(text)
    values = [(x,analyzer.polarity_scores(x)) for x in sentences]
    
    polarity = ['','']
    sentence = ['','']
    
    pos_polarity = [values[x][1]['pos'] for x in range(int(len(values)))]
    polarity[0] = max(pos_polarity)
    x = search(values, polarity[0], 'pos')
    sentence[0] = values[x]
    
    neg_polarity = [values[x][1]['neg'] for x in range(int(len(values)))]
    polarity[1] = max(neg_polarity)
    x = search(values, polarity[1],'neg')
    sentence[1] = values[x]
    
    return sentence

sentences = q3func(text)


In [163]:
sentences

[('“It’s so clever.',
  {'neg': 0.0, 'neu': 0.378, 'pos': 0.622, 'compound': 0.5095}),
 ('No one would expect KFC to bring out a collectible handbag.',
  {'neg': 0.196, 'neu': 0.804, 'pos': 0.0, 'compound': -0.296})]